In [1]:
!pip install -r requirements.txt

In [2]:
import requests
from bs4 import BeautifulSoup
import time
import random
import re
import json
import pandas as pd
from tqdm.notebook import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
import gensim
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

In [5]:
def scrape_wikipedia_article(article_title):
    """Mengambil judul dan konten teks utama dari sebuah artikel Wikipedia."""
    url = f"https://id.wikipedia.org/wiki/{article_title.replace(' ', '_')}"
    #Ini hanyalah contoh,ganti dengan URL yang anda inginkan
    
    try:
        headers = {'User-Agent': 'Custom NLP Crawler 1.0'}
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status() 
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Selektor spesifik Wikipedia untuk konten utama
        title = soup.find('h1', {'id': 'firstHeading'}).get_text(strip=True)
        content_div = soup.find('div', {'id': 'mw-content-text'})
        
        paragraphs = content_div.find_all('p') if content_div else []
        
        full_text = []
        for p in paragraphs:
            text = p.get_text(strip=True)
            # Menghilangkan referensi [1], [2], dll.
            text = re.sub(r'\[\d+\]', '', text)
            
            if len(text) > 100: # Filter paragraf yang terlalu pendek
                full_text.append(text)
        
        context = ' '.join(full_text)
        context = re.sub(r'\s+', ' ', context).strip()
        
        return title, context
    
    except requests.RequestException as e:
        # print(f"Gagal mengambil URL {url}: {e}")
        return None, None
    except Exception as e:
        # print(f"Error pemrosesan HTML {url}: {e}")
        return None, None

article_titles = [
    "Ekonomi Indonesia", "Politik Indonesia", "Budaya Indonesia",
    "Sejarah Jakarta", "Sepak bola di Indonesia", "Gunung di Indonesia",
    "Pariwisata di Bali", "Bahasa Indonesia", "Hewan endemik Indonesia",
    "Presiden Indonesia", "Sistem pemerintahan Indonesia"
    
]

scraped_data = []
for title in tqdm(article_titles, desc="Scraping Articles"):
    article_title, context = scrape_wikipedia_article(title)
    
    if context and len(context) > 500: # Minimum panjang konteks
        scraped_data.append({
            "title": article_title,
            "context": context
        })
        
    
    time.sleep(random.uniform(1.5, 3)) 

# Simpan ke JSON
output_file = "contexts.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(scraped_data, f, ensure_ascii=False, indent=4)
    
print(f"\nSelesai. Total {len(scraped_data)} artikel disimpan di {output_file}")
df = pd.DataFrame(scraped_data)

Scraping Articles:   0%|          | 0/11 [00:00<?, ?it/s]


Selesai. Total 8 artikel disimpan di contexts.json


In [12]:
#Preprocessing
stop_words_id = [
    'yang', 'dan', 'di', 'dari', 'itu', 'ini', 'adalah', 'dengan', 'sebagai', 
    'pada', 'untuk', 'tidak', 'atau', 'akan', 'telah', 'serta', 'namun', 
    # Tambahkan daftar stop words Indonesia yang lebih lengkap di sini
]

def preprocess_text(text):
    # 1. Lowercasing
    text = text.lower()
    # 2. non-alfanumerik
    text = re.sub(r'[^a-z\s]', '', text)
    # 3. Tokenisasi
    tokens = text.split()
    # 4. Hapus stop words
    tokens = [word for word in tokens if word not in stop_words_id and len(word) > 2]
    # 5. Gabungkan kembali (untuk scikit-learn)
    return ' '.join(tokens)

df['clean_context'] = df['context'].apply(preprocess_text)
print("Data setelah preprocessing:")
print(df[['title', 'clean_context']].head())

vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words=stop_words_id)
dtm = vectorizer.fit_transform(df['clean_context'])

#Pemodelan LDA
n_topics = 5 # jumlah topik
lda_model = LDA(n_components=n_topics, random_state=42, max_iter=10)
lda_model.fit(dtm)

# Fungsi untuk menampilkan kata-kata di setiap topik
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = f"Topik #{topic_idx + 1}: "
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-no_top_words - 1:-1]])
        print(message)

print("\nHasil Pemodelan Topik (LDA):")
display_topics(lda_model, vectorizer.get_feature_names_out(), no_top_words=10)

Data setelah preprocessing:
                     title                                      clean_context
0        Ekonomi Indonesia  ekonomiindonesiamerupakan ekonomi terbesar dia...
1        Politik Indonesia  politik indonesiaadalah berlangsung dalam rang...
2         Budaya Indonesia  kebudayaan indonesiaadalah seluruh kebudayaan ...
3          Sejarah Jakarta  jakartaadalah ibu kota kota terbesarindonesia ...
4  Sepak bola di Indonesia  sepak bolaadalah salah satu olahraga paling po...

Hasil Pemodelan Topik (LDA):
Topik #1: bahasa presiden wakil melayu konstitusi kepresidenan republik kata lembaga ris
Topik #2: bola kebudayaan sunda pelabuhan nasional daerah kota budaya bangsa merupakan
Topik #3: republik rakyat pemilihan presiden kekuasaan politik anggota umum daerah perwakilan
Topik #4: ekonomi keuangan krisis mencapai publik anggaran kebijakan lebih hal sektor
Topik #5: bali pulau sampai mereka asal tiongkok juga pelayaran sektor para


In [8]:
# Menetapkan Topik Dominan
topic_results = lda_model.transform(dtm)
# Cari indeks topik dengan probabilitas tertinggi di setiap dokumen
df['dominant_topic_index'] = topic_results.argmax(axis=1)

# Tambahkan probabilitas topik dominan
df['topic_probability'] = topic_results.max(axis=1)

# Asumsi: Anda sudah menginterpretasikan dan memberi nama pada topik (misalnya Topik 0 = Ekonomi)
topic_names = {
    0: "Ekonomi & Finansial", 
    1: "Politik & Pemerintahan", 
    2: "Budaya & Seni", 
    3: "Pariwisata & Geografi",
    4: "Bahasa & Linguistik" 
}
df['dominant_topic_name'] = df['dominant_topic_index'].map(topic_names)

print("Klasifikasi Otomatis Dokumen:")
print(df[['title', 'dominant_topic_name', 'topic_probability']])

Klasifikasi Otomatis Dokumen:
                     title     dominant_topic_name  topic_probability
0        Ekonomi Indonesia   Pariwisata & Geografi           0.939604
1        Politik Indonesia           Budaya & Seni           0.937469
2         Budaya Indonesia  Politik & Pemerintahan           0.931467
3          Sejarah Jakarta  Politik & Pemerintahan           0.908029
4  Sepak bola di Indonesia  Politik & Pemerintahan           0.903027
5       Pariwisata di Bali     Bahasa & Linguistik           0.896757
6         Bahasa Indonesia     Ekonomi & Finansial           0.885739
7       Presiden Indonesia     Ekonomi & Finansial           0.894258


In [9]:
# 1. Tokenisasi ulang untuk format Gensim
processed_texts = [doc.split() for doc in df['clean_context']]

# 2. Membuat Dictionary dan Corpus (format yang dibutuhkan Gensim)
dictionary = Dictionary(processed_texts)
corpus = [dictionary.doc2bow(text) for text in processed_texts]

# 3. Konversi model Scikit-learn LDA ke format Gensim LDA (atau latih ulang menggunakan Gensim)
lda_gensim = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=n_topics,
    random_state=42,
    chunksize=100,
    passes=10,
    per_word_topics=True
)

# 4. Hitung Coherence Score (Metrik C_v adalah yang paling umum)
from gensim.models.coherencemodel import CoherenceModel

coherence_model_lda = CoherenceModel(
    model=lda_gensim, 
    texts=processed_texts, 
    dictionary=dictionary, 
    coherence='c_v'
)

coherence_score = coherence_model_lda.get_coherence()

# Tampilkan Matriks Pengukuran
print("="*50)
print("HASIL EVALUASI MATRIKS PENGUKURAN")
print(f"1. Metode Evaluasi: Coherence Score (C_v)")
print(f"2. Coherence Score (C_v) Model: {coherence_score:.4f}")
print(f"Interpretasi: Semakin tinggi nilainya (mendekati 1.0), semakin baik model Anda menemukan topik yang masuk akal.")
print("="*50)

HASIL EVALUASI MATRIKS PENGUKURAN
1. Metode Evaluasi: Coherence Score (C_v)
2. Coherence Score (C_v) Model: 0.2631
Interpretasi: Semakin tinggi nilainya (mendekati 1.0), semakin baik model Anda menemukan topik yang masuk akal.


In [10]:
# Fungsi Testing

def preprocess_text_for_test(text):
    return preprocess_text(text)

In [11]:
import numpy as np

# Teks input baru (harus relevan dengan salah satu topik yang Anda latih)
new_texts = [
    # Teks 1: Ekonomi (Prediksi: Topik 1)
    "Bank Indonesia menaikkan suku bunga acuan 25 basis poin untuk mengendalikan inflasi dan menstabilkan nilai tukar rupiah terhadap dolar Amerika.", 
    
    # Teks 2: Politik (Prediksi: Topik 2)
    "Dewan Perwakilan Rakyat (DPR) menyetujui Rancangan Undang-Undang baru setelah melalui proses voting yang panjang dan alot di sidang paripurna.",
    
    # Teks 3: Budaya (Prediksi: Topik 3)
    "Pertunjukan wayang kulit dengan dalang ternama menarik ribuan penonton, membuktikan seni tradisi masih diminati generasi muda."
]

# Mapping nama topik (gunakan mapping dari Cell 4)
topic_names = {
    0: "Ekonomi & Finansial", 
    1: "Politik & Pemerintahan", 
    2: "Budaya & Seni", 
    3: "Pariwisata & Geografi",
    4: "Bahasa & Linguistik" 
}
n_topics = len(topic_names)


print("="*60)
print("HASIL INFERENSI MODEL LDA PADA DOKUMEN BARU")
print("="*60)

for i, text in enumerate(new_texts):
    # 1. Preprocessing
    clean_text = preprocess_text_for_test(text)
    
    # 2. Vektorisasi
    dtm_new = vectorizer.transform([clean_text])
    
    # 3. Prediksi Probabilitas Topik
    topic_probabilities = lda_model.transform(dtm_new)[0]
    
    # 4. Menemukan Topik Dominan
    dominant_topic_index = np.argmax(topic_probabilities)
    dominant_topic_prob = topic_probabilities[dominant_topic_index]
    
    print(f"--- [TEST {i+1}] ---")
    print(f"Input: {text[:80]}...")
    print(f"Topik Ditemukan (Index): Topik #{dominant_topic_index}")
    print(f"Nama Topik Dominan: {topic_names.get(dominant_topic_index, 'Tidak Dikenal')}")
    print(f"Probabilitas Dominan: {dominant_topic_prob:.4f}")
    
    # (Opsional) Tampilkan semua probabilitas
    print("\nProbabilitas Semua Topik:")
    for topic_idx in range(n_topics):
        print(f"  - {topic_names.get(topic_idx)}: {topic_probabilities[topic_idx]:.4f}")
    print("-" * 60)

HASIL INFERENSI MODEL LDA PADA DOKUMEN BARU
--- [TEST 1] ---
Input: Bank Indonesia menaikkan suku bunga acuan 25 basis poin untuk mengendalikan infl...
Topik Ditemukan (Index): Topik #3
Nama Topik Dominan: Pariwisata & Geografi
Probabilitas Dominan: 0.7226

Probabilitas Semua Topik:
  - Ekonomi & Finansial: 0.0688
  - Politik & Pemerintahan: 0.0715
  - Budaya & Seni: 0.0685
  - Pariwisata & Geografi: 0.7226
  - Bahasa & Linguistik: 0.0686
------------------------------------------------------------
--- [TEST 2] ---
Input: Dewan Perwakilan Rakyat (DPR) menyetujui Rancangan Undang-Undang baru setelah me...
Topik Ditemukan (Index): Topik #2
Nama Topik Dominan: Budaya & Seni
Probabilitas Dominan: 0.7943

Probabilitas Semua Topik:
  - Ekonomi & Finansial: 0.0521
  - Politik & Pemerintahan: 0.0512
  - Budaya & Seni: 0.7943
  - Pariwisata & Geografi: 0.0512
  - Bahasa & Linguistik: 0.0511
------------------------------------------------------------
--- [TEST 3] ---
Input: Pertunjukan wayang k